### Module Importation and DataFrame Setup

In [1]:
# Reset the model for re-run
from tensorflow.keras import backend as K
K.clear_session()

# Import dependencies
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Read the preprocessed CSV file from the Dataset folder into a Pandas DataFrame (see logistic regression model for preprocessing)
dfSurvivals = pd.read_csv(
    Path("../CSV_OUTPUT/ML_Cleaned_DS_CSV.csv")
)

In [3]:
# Review the DataFrame
dfSurvivals.head()

,Passenger Id,Age,Sibling/Spouse Abroad,Parent/Children Abroad,Passenger Class,Fare,Embarkation Port,Survival Boat,Body Number,Survived,Female,Male
0,1,29.0000,0,0,1,211.3375,2,2,S,1,1.0,0.0
1,2,0.9167,1,2,1,151.5500,2,11,S,1,0.0,1.0
2,3,2.0000,1,2,1,151.5500,2,DNS,BNR,0,1.0,0.0
3,4,30.0000,1,2,1,151.5500,2,DNS,135.0,0,0.0,1.0
4,5,25.0000,1,2,1,151.5500,2,DNS,BNR,0,1.0,0.0


In [4]:
# Print DataFrame information to ensure no null values (data cleaned in previous notebook) and check datatypes
dfSurvivals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308 entries, 0 to 1307
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Passenger Id            1308 non-null   int64  
 1   Age                     1308 non-null   float64
 2   Sibling/Spouse Abroad   1308 non-null   int64  
 3   Parent/Children Abroad  1308 non-null   int64  
 4   Passenger Class         1308 non-null   int64  
 5   Fare                    1308 non-null   float64
 6   Embarkation Port        1308 non-null   int64  
 7   Survival Boat           1308 non-null   object 
 8   Body Number             1308 non-null   object 
 9   Survived                1308 non-null   int64  
 10  Female                  1308 non-null   float64
 11  Male                    1308 non-null   float64
dtypes: float64(4), int64(6), object(2)
memory usage: 122.8+ KB


In [5]:
# Drop Passenger ID, Survival Boat, and Body Number columns
dfSurvivals = dfSurvivals.drop(['Passenger Id', 'Survival Boat', 'Body Number'], axis=1)
dfSurvivals.head()

,Age,Sibling/Spouse Abroad,Parent/Children Abroad,Passenger Class,Fare,Embarkation Port,Survived,Female,Male
0,29.0000,0,0,1,211.3375,2,1,1.0,0.0
1,0.9167,1,2,1,151.5500,2,1,0.0,1.0
2,2.0000,1,2,1,151.5500,2,0,1.0,0.0
3,30.0000,1,2,1,151.5500,2,0,0.0,1.0
4,25.0000,1,2,1,151.5500,2,0,1.0,0.0


### Split the data into X and y and then into testing and training sets

In [7]:
# Split the data into X (features) and y (target)

# Set the y variable, which is the target
y = dfSurvivals['Survived']

# Set the X variable, which includes all features escept the target
X = dfSurvivals.drop(columns=['Survived'])

In [8]:
# Split into testing and training sets using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Compile, Train, and Evaluate the Model

In [10]:
# Get the number of input features
input_dim = X_train.shape[1]

# Function to create and compile the neural network
def create_model(hp):
    
    # Create a new model
    nn_model = tf.keras.models.Sequential()
    
    # Choose activation function for hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Add first hidden layer with hyperparameter-tuned settings
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units', 
        min_value=1, 
        max_value=10, 
        step=2), 
        activation=activation,
        
        # Removed L2 regularization
        # kernel_regularizer=tf.keras.regularizers.l2(0.01),
        input_dim=input_dim))
    
    # Added Dropout layer to prevent overfitting
    nn_model.add(tf.keras.layers.Dropout(0.5))
    
    # Add second hidden layer with 10 neurons
    nn_model.add(tf.keras.layers.Dense(units=10, activation=activation))
    
    # Add output layer for binary classification
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    
    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy", "Precision", "Recall"]) 
    
    # Return compiled model
    return nn_model

In [12]:
# Set up the kerastuner then allow it to search for best hyperparameters
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2,
    overwrite=True)

# Initialize EarlyStopping callback to monitor the validation loss
# Training will stop if the validation loss doesn't improve for 3 consecutive epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Perform hyperparameter search
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test), callbacks=[early_stopping])

Trial 20 Complete [00h 00m 02s]
val_accuracy: 0.7003058195114136

Best val_accuracy So Far: 0.7798165082931519
Total elapsed time: 00h 00m 39s
INFO:tensorflow:Oracle triggered exit


In [13]:
# Get the hyperparameters of the best model
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 9,
 'tuner/epochs': 7,
 'tuner/initial_epoch': 3,
 'tuner/bracket': 2,
 'tuner/round': 1,
 'tuner/trial_id': '0010'}

In [14]:
# Get the best model
best_model = tuner.get_best_models(1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 81        
                                                                 
 dropout (Dropout)           (None, 9)                 0         
                                                                 
 dense_1 (Dense)             (None, 10)                100       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 192 (768.00 Byte)
Trainable params: 192 (768.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Train the model
fit_model = best_model.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
31/31 [==============================] - 1s 1ms/step - loss: 0.5583 - accuracy: 0.7105 - precision: 0.6754 - recall: 0.4788
Epoch 2/100
31/31 [==============================] - 0s 1ms/step - loss: 0.5529 - accuracy: 0.7350 - precision: 0.7049 - recall: 0.5370
Epoch 3/100
31/31 [==============================] - 0s 1ms/step - loss: 0.5541 - accuracy: 0.7217 - precision: 0.6966 - recall: 0.4921
Epoch 4/100
31/31 [==============================] - 0s 1ms/step - loss: 0.5499 - accuracy: 0.7085 - precision: 0.6704 - recall: 0.4788
Epoch 5/100
31/31 [==============================] - 0s 1ms/step - loss: 0.5463 - accuracy: 0.7105 - precision: 0.6610 - recall: 0.5106
Epoch 6/100
31/31 [==============================] - 0s 1ms/step - loss: 0.5443 - accuracy: 0.7258 - precision: 0.6926 - recall: 0.5185
Epoch 7/100
31/31 [==============================] - 0s 1ms/step - loss: 0.5531 - accuracy: 0.7176 - precision: 0.6810 - recall: 0.5026
Epoch 8/100
31/31 [=============================

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_precision, model_recall = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}, Precision: {model_precision}, Recall: {model_recall}")

11/11 - 0s - loss: 0.4734 - accuracy: 0.7982 - precision: 0.7979 - recall: 0.6148 - 229ms/epoch - 21ms/step
Loss: 0.4734460711479187, Accuracy: 0.7981651425361633, Precision: 0.7978723645210266, Recall: 0.6147540807723999


In [17]:
# Use the model to make predictions on test data
y_pred_prob = best_model.predict(X_test_scaled).flatten()
y_pred = [1 if prob >= 0.5 else 0 for prob in y_pred_prob]

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Generate a classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

11/11 [==============================] - 0s 901us/step
Confusion Matrix:
[[186  19]
 [ 47  75]]
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.91      0.85       205
           1       0.80      0.61      0.69       122

    accuracy                           0.80       327
   macro avg       0.80      0.76      0.77       327
weighted avg       0.80      0.80      0.79       327



In [19]:
# Export our model to keras file
best_model.save('OUTPUT/Titanic_survival_mod1_opt2.keras')